In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import numpy
%matplotlib inline

In [32]:
# import the whole dataset
points_m_from_2017 = pd.read_csv(r"C:\Users\carlo\TrainingsTool\Tennis\Datasets\charting-m-points-from-2017-new.csv", encoding= 'unicode_escape', low_memory=False)

In [41]:
points_m_from_2017.loc[:, ["Pts", "Gm1", "Gm2", "Set1", "Set2", "Serving"]].head(20)

,Pts,Gm1,Gm2,Set1,Set2,Serving
0,0-0,0,0,0,0,MC
1,0-15,0,0,0,0,MC
2,15-15,0,0,0,0,MC
3,15-30,0,0,0,0,MC
4,30-30,0,0,0,0,MC
5,40-30,0,0,0,0,MC
6,40-40,0,0,0,0,MC
7,AD-40,0,0,0,0,MC
8,40-40,0,0,0,0,MC
9,40-AD,0,0,0,0,MC


In [18]:
# only take matches where a specific player played
djokovic_points_from_2017 = points_m_from_2017.loc[points_m_from_2017["match_id"].str.contains("Novak_Djokovic", case = True)]


In [17]:
djokovic_points_from_2017.size

579720

In [44]:
# take the relevant columns from the points where djokovic played
test_df = djokovic_points_from_2017.loc[:, ["Pts", "Gm1", "Gm2", "Set1", "Set2", "Serving", "1st", "2nd"]].head(20)

In [46]:
# ralley colums are being converted into strings
test_df = test_df.astype({"1st": str, "2nd": str})

In [47]:
# join the two ralley colums together into one column with "," seperator
test_df["1st,2nd"] = test_df[["1st", "2nd"]].apply(lambda x: ",".join(x), axis=1)
test_df.head()

,Pts,Gm1,Gm2,Set1,Set2,Serving,1st,2nd,"1st,2nd"
637,0-0,0,0,0,0,ND,4n,4f18f3s2f1f3b3n@,"4n,4f18f3s2f1f3b3n@"
638,0-15,0,0,0,0,ND,4n,6f2n#,"4n,6f2n#"
639,15-15,0,0,0,0,ND,4*,nan,"4*,nan"
640,30-15,0,0,0,0,ND,4n,5b3n@,"4n,5b3n@"
641,40-15,0,0,0,0,ND,4r28f1r2f3b3f2f1f1f3b3b3b1r2n#,nan,"4r28f1r2f3b3f2f1f1f3b3b3b1r2n#,nan"


In [48]:
# drop the two other columns which are irrelevant now
test_df.drop(["1st", "2nd"], axis = 1, inplace=True)
test_df.head()

,Pts,Gm1,Gm2,Set1,Set2,Serving,"1st,2nd"
637,0-0,0,0,0,0,ND,"4n,4f18f3s2f1f3b3n@"
638,0-15,0,0,0,0,ND,"4n,6f2n#"
639,15-15,0,0,0,0,ND,"4*,nan"
640,30-15,0,0,0,0,ND,"4n,5b3n@"
641,40-15,0,0,0,0,ND,"4r28f1r2f3b3f2f1f1f3b3b3b1r2n#,nan"


In [131]:
ralley_df = test_df["1st,2nd"].head(1)
ralley_df

637    4n,4f18f3s2f1f3b3n@
Name: 1st,2nd, dtype: object

In [132]:
# now comes the tricky bit: each tennis shot has to be its own column to make predictions on that

# go through each character in the "1st,2nd" (ralley) column and follow the schema to add the new columns
# if the next char in the ralley is in the list of possible_shots
# then add the chars until that char as a new shot in a new column

possible_shots = ["f", "b", "r", "s", "v", "z", "o", "p", "y", "l", "m", "h", "i", "j", "k", "t", "u"]
second_serve = ","

# i is a ralley
for i in ralley_df:
    # Initializeing new shot for each new ralley
    shot = ""
    shot_list = []
    # every char in each ralley will be looked at
    for char in range(0, len(i)):
        # if the char is not in the possible_shots list, the char is appended to the current shot
        if i[char] not in possible_shots and i[char] != second_serve:
            shot = shot + str(i[char])
        # if the char is the ",", shot is done and appended to the shot_list
        elif i[char] == second_serve:
            shot = shot + second_serve
            shot_list.append(shot)
            shot = ""
        # in all other cases, shot is done and appended to the shot_list
        else:
            shot_list.append(shot)
            shot = str(i[char])
    # the last shot has to be appended here, because it is not going into the loop again
    shot_list.append(shot)

    # if the last element of the shot list is "nan" we delete that because we dont want wo predict "nan" and also the "," from the last shot
    if shot_list[-1] == "nan":
        shot_list.pop()
        shot_list[-1] = shot_list[-1].rstrip(shot_list[-1][-1])
    
    print(shot_list)
    # now the shot list is beautiful and we can add each element of it as a new column in the big dataframe
    print(len(shot_list))
    for shot in range(len(shot_list)):
        name = "ralley_shot_" + str(shot)
        
    
    print(ralley_df)



SyntaxError: invalid syntax. Maybe you meant '==' or ':=' instead of '='? (2986521125.py, line 41)

In [ ]:
# now seperate into serving and returning datasets